# Аугментация данных с помощью порождающих грамматик

В данном файле генерируются команды, не соответствующие установленной форме произношения, которые в том числе могут быть произнесены случайно

In [9]:
GRAMMAR = """
COMMAND -> PREFIX CMD POSTFIX | CMD PREFIX POSTFIX | PREFIX POSTFIX CMD 
CMD -> SINGLE [0.2] | MULTIPLE [0.6] | RANDOM [0.2]

RANDOM -> SINGLE | CMD1 | CMD2 | CMD3 | CMD4 | CMD5 | CMD6 | CMD7 | CMD8
RANDOM -> CMD9 | CMD10 | CMD11 | CMD12 | CMD13 | CMD14 | CMD15
RANDOM -> RANDOM [0.6] | CURSE RANDOM [0.1]

MULTIPLE -> CMD1 CMD1 | CMD2 CMD2 CMD2 CMD2 | CMD3 CMD3 | CMD4 CMD4 CMD4
MULTIPLE -> CMD5 CMD5 CMD5 CMD5 | CMD6 CMD6 | CMD7 CMD7 | CMD8 CMD8 CMD8 CMD8
MULTIPLE -> CMD9 CMD9 CMD9 | CMD10 CMD10 CMD10 CMD10 | CMD11 CMD11 CMD11 | CMD12 CMD12
MULTIPLE -> CMD13 CMD13 CMD13 | CMD14 CMD14 CMD14 | CMD15 CMD15 CMD15 CMD15

SINGLE -> 'отказ' | 'отмена' | 'подтверждение' | 'отцепка' | 'остановка' | 'тише' | 'тормозить' | 'отпустить'
CMD1 -> 'начать' [0.5] | 'осаживание' [0.5]
CMD2 -> 'осадить' | 'на' | 'вагон' | NUM
CMD3 -> 'продолжаем' | 'освежевание'
CMD4 -> 'зарядка' | 'тормозной' | 'магистрали'
CMD5 -> 'вышел' | 'из' | 'межвагонного' | 'пространства'
CMD6 -> 'продолжаем' | 'роспуск'
CMD7 -> 'растянуть' | 'автосцепки'
CMD8 -> 'протянуть' | 'на' | NUM | 'вагон'
CMD9 -> 'назад' | 'на' | 'башмак'
CMD10 -> 'захожу' | 'в' | 'межвагонное' | 'пространство'
CMD11 -> 'вперёд' | 'на' | 'башмак'
CMD12 -> 'сжать' | 'автосцепки'
CMD13 -> 'назад' | 'с' | 'башмака'
CMD14 -> 'вперед' | 'с' | 'башмака'
CMD15 -> 'прекратить' | 'зарядку' | 'тормозной' | 'магистрали' 
PREFIX -> 'э' | 'хмм' | 'ой' | 'я' | 'ты' | CURSE

CURSE -> 'блин' | 'чёрт' | 'блин' | 'ёмаё' | 'короче' | 'типа' | 'впринципе' | 'а' | 'аа' | 'ааа' | 'значит'
POSTFIX -> PREFIX | 'вот' | 'так' | 'вот так'

NUM -> 'ноль'
NUM -> 'один'
NUM -> 'два'
NUM -> 'три'
NUM -> 'четыре'
NUM -> 'пять'
NUM -> 'шесть'
NUM -> 'семь'
NUM -> 'восемь'
NUM -> 'девять'
NUM -> 'десять'
NUM -> 'одиннадцать'
NUM -> 'двенадцать'
NUM -> 'тринадцать'
NUM -> 'четырнадцать'
NUM -> 'пятнадцать'
NUM -> 'шестнадцать'
NUM -> 'семнадцать'
NUM -> 'восемнадцать'
NUM -> 'девятнадцать'
NUM -> 'двадцать'
NUM -> 'тридцать'
NUM -> 'сорок'
NUM -> 'пятьдесят'
NUM -> 'шестьдесят'
NUM -> 'семьдесят'
NUM -> 'восемьдесят'
NUM -> 'девяносто'
NUM -> 'сто'
"""

In [10]:
from nltk import read_grammar
from nltk.grammar import standard_nonterm_parser, ProbabilisticProduction

start, productions = read_grammar(GRAMMAR, standard_nonterm_parser, probabilistic=True, encoding=None)


In [11]:
# Автоматический подсчёт вероятностей, чтобы в сумме
# для нетерминалов, где вероятность не объявлена, сумма равнялась 1

tmp = {}

for prod in productions:
    lhs = prod.lhs()
    prob = prod.prob()
    if prob == 0:
        rhss, count, defined_prob = tmp.get(lhs, ([], 0, 0))
        rhss.append(prod)
        tmp[lhs] = (rhss, count + 1, defined_prob)
    else:
        rhss, count, defined_prob = tmp.get(lhs, ([], 0, 0))
        tmp[lhs] = (rhss, count + 1, defined_prob + prob)

new_productions = [p for p in productions if p.prob() != 0]

for lhs, (prods, count, defined_prob) in tmp.items():
    p = (1 - defined_prob) / count
    for prod in prods:
        new_productions.append(ProbabilisticProduction(prod.lhs(), prod.rhs(), prob=p))

new_productions

[CMD -> SINGLE [0.2],
 CMD -> MULTIPLE [0.6],
 CMD -> RANDOM [0.2],
 RANDOM -> RANDOM [0.6],
 RANDOM -> CURSE RANDOM [0.1],
 CMD1 -> 'начать' [0.5],
 CMD1 -> 'осаживание' [0.5],
 COMMAND -> PREFIX CMD POSTFIX [0.333333],
 COMMAND -> CMD PREFIX POSTFIX [0.333333],
 COMMAND -> PREFIX POSTFIX CMD [0.333333],
 RANDOM -> SINGLE [0.0166667],
 RANDOM -> CMD1 [0.0166667],
 RANDOM -> CMD2 [0.0166667],
 RANDOM -> CMD3 [0.0166667],
 RANDOM -> CMD4 [0.0166667],
 RANDOM -> CMD5 [0.0166667],
 RANDOM -> CMD6 [0.0166667],
 RANDOM -> CMD7 [0.0166667],
 RANDOM -> CMD8 [0.0166667],
 RANDOM -> CMD9 [0.0166667],
 RANDOM -> CMD10 [0.0166667],
 RANDOM -> CMD11 [0.0166667],
 RANDOM -> CMD12 [0.0166667],
 RANDOM -> CMD13 [0.0166667],
 RANDOM -> CMD14 [0.0166667],
 RANDOM -> CMD15 [0.0166667],
 MULTIPLE -> CMD1 CMD1 [0.0666667],
 MULTIPLE -> CMD2 CMD2 CMD2 CMD2 [0.0666667],
 MULTIPLE -> CMD3 CMD3 [0.0666667],
 MULTIPLE -> CMD4 CMD4 CMD4 [0.0666667],
 MULTIPLE -> CMD5 CMD5 CMD5 CMD5 [0.0666667],
 MULTIPLE -> CMD

In [12]:
from nltk.grammar import PCFG

PCFG.EPSILON = 0.05

from pcfg import PCFG

grammar = PCFG(start, new_productions)

In [13]:
transcriptions = []

for sentence in grammar.generate(1000):
    transcriptions.append(sentence)

print(len(transcriptions))

1000


In [14]:
from pandas import DataFrame

df = DataFrame.from_dict({'transcriptions': transcriptions, 'label': [-1 for _ in range(len(transcriptions))]})
df.to_csv("generated.csv", index=False)
df

,transcriptions,label
0,осаживание начать ой так,-1
1,я вот магистрали магистрали тормозной,-1
2,ой отказ так,-1
3,ёмаё так автосцепки сжать,-1
4,ой так магистрали тормозной тормозной,-1
...,...,...
995,я я освежевание,-1
996,ты осаживание осаживание так,-1
997,ой э автосцепки,-1
998,ты так начать начать,-1
